In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from skyfield.api import Star, load
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
from skyfield.data import hipparcos, mpc, stellarium
from skyfield.projections import build_stereographic_projection

# The comet is plotted on several dates `t_comet`.  But the stars only
# need to be drawn once, so we take the middle comet date as the single
# time `t` we use for everything else.

ts = load.timescale()
t = ts.utc(2020, 10, 26)

# The Hipparcos mission provides our star catalog.

with load.open(hipparcos.URL) as f:
    stars = hipparcos.load_dataframe(f)

# We will center the chart on the comet's middle position.

from skyfield.positionlib import ICRS, Barycentric
barycenter = Barycentric([0,0,0], t=t)
center = ICRS([1,1,1])
projection = build_stereographic_projection(center)
field_of_view_degrees = 45.0
limiting_magnitude = 7.0

# Create a True/False mask marking the stars bright enough to be
# included in our plot.  And go ahead and compute how large their
# markers will be on the plot.

from time import time
print('Total stars:', stars.shape)
t0 = time()
bright_stars = (stars.magnitude <= limiting_magnitude)
print(time() - t0, 's to filter all stars on 1 var')

magnitude = stars['magnitude'][bright_stars]
marker_size = (0.5 + limiting_magnitude - magnitude) ** 2.0

# Now that we have constructed our projection, compute the x and y
# coordinates that each star and the comet will have on the plot.

star_positions = barycenter.observe(Star.from_dataframe(stars))
t0 = time()
stars['x'], stars['y'] = projection(star_positions)
print(time() - t0)

# Time to build the figure!

fig, ax = plt.subplots(figsize=[9, 9])

# Draw the stars.

mask = (
    bright_stars
    & (stars['x'] > -0.3)
    & (stars['x'] < 0.3)
    & (stars['y'] > -0.3)
    & (stars['y'] < 0.3)
)

print('Number of stars:', mask.sum())

ax.scatter(stars['x'][mask], stars['y'][mask],
           s=marker_size[mask[bright_stars]], color='k')

# Finally, title the plot and set some final parameters.

angle = np.pi - field_of_view_degrees / 360.0 * np.pi
limit = np.sin(angle) / (1.0 - np.cos(angle))
#print(limit)
ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_aspect(1.0)
ax.set_title('Stars')

from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox, interact
from ipyevents import Event 
from IPython.display import display

l = Label('Click or type on me!')
l.layout.border = '2px solid red'

h = HTML('Event info')
d = Event(source=l, watched_events=['click', 'keydown', 'mouseenter'])

def handle_event(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    h.value = content

d.on_dom_event(handle_event)
                            
#display(l, h)

import ipywidgets as widgets

def demo(i):
    field_of_view_degrees = 45.0 - i
    angle = np.pi - field_of_view_degrees / 360.0 * np.pi
    limit = np.sin(angle) / (1.0 - np.cos(angle))

    ax.set_xlim(-limit, limit)
    ax.set_ylim(-limit, limit)
    return fig

#widgets.interact(demo, i = d)

from io import BytesIO
io = BytesIO()
plt.savefig(io, format="png")

image = Image(value=io.getvalue(), format='png')

# The layout bits below make sure the image display looks the same in lab and classic notebook 
image.layout.max_width = '100%'
image.layout.height = 'auto'

im_events = Event()
im_events.source = image
im_events.watched_events = ['click']

image

In [ ]:
import ipywidgets as widgets
from IPython.display import HTML

In [ ]:
def demo(i):
    field_of_view_degrees = 45.0 - i
    angle = np.pi - field_of_view_degrees / 360.0 * np.pi
    limit = np.sin(angle) / (1.0 - np.cos(angle))

    ax.set_xlim(-limit, limit)
    ax.set_ylim(-limit, limit)
    return fig

widgets.interact(demo, i = widgets.Play(
    value=0,
    min=0,
    max=30,
    step=1,
    description="Press play",
    disabled=False))
None

In [ ]:
play = widgets.Play(
    value=50,
    min=0,
    max=100,
    step=1,
    interval=50,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

In [ ]:
w = widgets.HTML(
    value="Hello <b>World</b>",
    placeholder='Some HTML',
    description='Some HTML',
)
def handle_text(sender):
    print("enter")
w.observe(handle_text, names='value')
w

In [ ]:
w = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
w.observe(handle_text)
w

In [ ]:
type(w)
dir(w)

In [ ]:
#%matplotlib inline
#%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import time

def pltsin(ax, colors=['b']):
    x = np.linspace(0,1,100)
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            y = np.random.random(size=(100,1))
            line.set_ydata(y)
    else:
        for color in colors:
            y = np.random.random(size=(100,1))
            ax.plot(x, y, color)
    fig.canvas.draw()

fig,ax = plt.subplots(1,1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
for f in range(5):
    pltsin(ax, ['b', 'r'])
    time.sleep(1)

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [ ]:
#%gui asyncio

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
from ipywidgets import IntSlider
slider = IntSlider()

async def f():
    for i in range(10):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

In [ ]:
#%matplotlib notebook

import sys
import numpy as np
import matplotlib.pyplot as plt


def on_press(event):
    print('press', event.key)
    sys.stdout.flush()
    if event.key == 'x':
        visible = xl.get_visible()
        xl.set_visible(not visible)
        fig.canvas.draw()
    return False

# Fixing random state for reproducibility
np.random.seed(19680801)


fig, ax = plt.subplots()

fig.canvas.mpl_connect('key_press_event', on_press)

ax.plot(np.random.rand(12), np.random.rand(12), 'go')
xl = ax.set_xlabel('easy come, easy go')
ax.set_title('Press a key')
plt.show()

In [ ]:
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox
from ipyevents import Event 
from IPython.display import display

In [ ]:
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox
from ipyevents import Event 
from IPython.display import display

l = Label('Click or type on me!')
l.layout.border = '2px solid red'

h = HTML('Event info')
d = Event(source=l, watched_events=['click', 'keydown', 'mouseenter'])

def handle_event(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    h.value = content

d.on_dom_event(handle_event)
                            
display(l, h)